In [1]:
import numpy as np
import scipy
import pandas as pd
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL

In [2]:
%matplotlib widget
plt.rcParams["figure.autolayout"] = True
plt.rcParams["figure.figsize"] = [7,5]

# Data Cleaning

In [3]:
os.getcwd()

'/Users/aprilnellis/Desktop/recession-regression/Data/Scripts'

In [6]:
original = pd.read_csv('../Raw/BLS_Employment.csv')
original

,Month,Total nonfarm,Total private,Mining and logging,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Utilities,Information,Financial activities,Professional and business services,Private education and health services,Leisure and hospitality,Other services,Government,Federal government,State government,Local government
0,9/1/2003,130252,108748,570,6783,14347,5537.4,14911.4,4162.1,573.1,3162,8100,16108,16883,12208,5403,21504,2749,4980,13775
1,10/1/2003,130252,108881,573,6784,14334,5537.1,14930.1,4171.3,573.0,3155,8090,16130,16942,12259,5402,21558,2742,4987,13829
2,11/1/2003,130252,108947,576,6796,14316,5539.2,14906.1,4174.8,571.5,3150,8083,16168,16978,12284,5404,21535,2728,4988,13819
3,12/1/2003,130252,109049,575,6827,14300,5551.4,14903.6,4161.6,570.8,3150,8076,16195,17016,12320,5403,21546,2740,4981,13825
4,1/1/2004,130252,109227,576,6848,14290,5562.8,14946.7,4191.7,570.2,3141,8078,16224,17048,12346,5405,21538,2732,4965,13841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,5/1/2023,155970,133341,643,7926,12981,6054.1,15548.1,6738.6,556.7,3091,9135,23017,25237,16557,5856,22629,2919,5216,14494
237,6/1/2023,156075,133427,642,7955,12985,6047.4,15525.4,6719.5,557.5,3081,9137,23016,25316,16583,5862,22648,2925,5231,14492
238,7/1/2023,156311,133572,644,7967,12983,6062.8,15538.0,6712.7,556.0,3062,9152,22987,25420,16621,5866,22739,2935,5243,14561
239,8/1/2023,156538,133749,644,8003,12994,6062.8,15538.4,6693.8,558.4,3041,9154,22998,25517,16665,5880,22789,2946,5267,14576


In [7]:
original['Month'] = pd.to_datetime(original['Month'])
original

,Month,Total nonfarm,Total private,Mining and logging,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Utilities,Information,Financial activities,Professional and business services,Private education and health services,Leisure and hospitality,Other services,Government,Federal government,State government,Local government
0,2003-09-01,130252,108748,570,6783,14347,5537.4,14911.4,4162.1,573.1,3162,8100,16108,16883,12208,5403,21504,2749,4980,13775
1,2003-10-01,130252,108881,573,6784,14334,5537.1,14930.1,4171.3,573.0,3155,8090,16130,16942,12259,5402,21558,2742,4987,13829
2,2003-11-01,130252,108947,576,6796,14316,5539.2,14906.1,4174.8,571.5,3150,8083,16168,16978,12284,5404,21535,2728,4988,13819
3,2003-12-01,130252,109049,575,6827,14300,5551.4,14903.6,4161.6,570.8,3150,8076,16195,17016,12320,5403,21546,2740,4981,13825
4,2004-01-01,130252,109227,576,6848,14290,5562.8,14946.7,4191.7,570.2,3141,8078,16224,17048,12346,5405,21538,2732,4965,13841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2023-05-01,155970,133341,643,7926,12981,6054.1,15548.1,6738.6,556.7,3091,9135,23017,25237,16557,5856,22629,2919,5216,14494
237,2023-06-01,156075,133427,642,7955,12985,6047.4,15525.4,6719.5,557.5,3081,9137,23016,25316,16583,5862,22648,2925,5231,14492
238,2023-07-01,156311,133572,644,7967,12983,6062.8,15538.0,6712.7,556.0,3062,9152,22987,25420,16621,5866,22739,2935,5243,14561
239,2023-08-01,156538,133749,644,8003,12994,6062.8,15538.4,6693.8,558.4,3041,9154,22998,25517,16665,5880,22789,2946,5267,14576


In [16]:
employ_diff = original.diff(axis = 0)
#employ_diff = pd.read_csv('BLS_EmployDelta.csv')
employ_diff

,Month,Total nonfarm,Total private,Mining and logging,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Utilities,Information,Financial activities,Professional and business services,Private education and health services,Leisure and hospitality,Other services,Government,Federal government,State government,Local government
0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30 days,0.0,133.0,3.0,1.0,-13.0,-0.3,18.7,9.2,-0.1,-7.0,-10.0,22.0,59.0,51.0,-1.0,54.0,-7.0,7.0,54.0
2,31 days,0.0,66.0,3.0,12.0,-18.0,2.1,-24.0,3.5,-1.5,-5.0,-7.0,38.0,36.0,25.0,2.0,-23.0,-14.0,1.0,-10.0
3,30 days,0.0,102.0,-1.0,31.0,-16.0,12.2,-2.5,-13.2,-0.7,0.0,-7.0,27.0,38.0,36.0,-1.0,11.0,12.0,-7.0,6.0
4,31 days,0.0,178.0,1.0,21.0,-10.0,11.4,43.1,30.1,-0.6,-9.0,2.0,29.0,32.0,26.0,2.0,-8.0,-8.0,-16.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,30 days,281.0,255.0,3.0,25.0,-4.0,7.7,20.9,18.6,0.7,-4.0,13.0,45.0,88.0,28.0,13.0,26.0,9.0,14.0,3.0
237,31 days,105.0,86.0,-1.0,29.0,4.0,-6.7,-22.7,-19.1,0.8,-10.0,2.0,-1.0,79.0,26.0,6.0,19.0,6.0,15.0,-2.0
238,30 days,236.0,145.0,2.0,12.0,-2.0,15.4,12.6,-6.8,-1.5,-19.0,15.0,-29.0,104.0,38.0,4.0,91.0,10.0,12.0,69.0
239,31 days,227.0,177.0,0.0,36.0,11.0,0.0,0.4,-18.9,2.4,-21.0,2.0,11.0,97.0,44.0,14.0,50.0,11.0,24.0,15.0


In [17]:
employ_diff.drop(0, axis=0, inplace = True)

In [18]:
employ_diff['Month'] = original['Month']
employ_diff

,Month,Total nonfarm,Total private,Mining and logging,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Utilities,Information,Financial activities,Professional and business services,Private education and health services,Leisure and hospitality,Other services,Government,Federal government,State government,Local government
1,2003-10-01,0.0,133.0,3.0,1.0,-13.0,-0.3,18.7,9.2,-0.1,-7.0,-10.0,22.0,59.0,51.0,-1.0,54.0,-7.0,7.0,54.0
2,2003-11-01,0.0,66.0,3.0,12.0,-18.0,2.1,-24.0,3.5,-1.5,-5.0,-7.0,38.0,36.0,25.0,2.0,-23.0,-14.0,1.0,-10.0
3,2003-12-01,0.0,102.0,-1.0,31.0,-16.0,12.2,-2.5,-13.2,-0.7,0.0,-7.0,27.0,38.0,36.0,-1.0,11.0,12.0,-7.0,6.0
4,2004-01-01,0.0,178.0,1.0,21.0,-10.0,11.4,43.1,30.1,-0.6,-9.0,2.0,29.0,32.0,26.0,2.0,-8.0,-8.0,-16.0,16.0
5,2004-02-01,0.0,45.0,1.0,-10.0,-11.0,5.7,-0.9,4.6,-1.5,0.0,0.0,13.0,18.0,26.0,0.0,12.0,-1.0,2.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2023-05-01,281.0,255.0,3.0,25.0,-4.0,7.7,20.9,18.6,0.7,-4.0,13.0,45.0,88.0,28.0,13.0,26.0,9.0,14.0,3.0
237,2023-06-01,105.0,86.0,-1.0,29.0,4.0,-6.7,-22.7,-19.1,0.8,-10.0,2.0,-1.0,79.0,26.0,6.0,19.0,6.0,15.0,-2.0
238,2023-07-01,236.0,145.0,2.0,12.0,-2.0,15.4,12.6,-6.8,-1.5,-19.0,15.0,-29.0,104.0,38.0,4.0,91.0,10.0,12.0,69.0
239,2023-08-01,227.0,177.0,0.0,36.0,11.0,0.0,0.4,-18.9,2.4,-21.0,2.0,11.0,97.0,44.0,14.0,50.0,11.0,24.0,15.0


In [19]:
employ_diff.iloc[:, 1:] = employ_diff.iloc[:, 1:]/original.iloc[:-1, 1:]
employ_diff

,Month,Total nonfarm,Total private,Mining and logging,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Utilities,Information,Financial activities,Professional and business services,Private education and health services,Leisure and hospitality,Other services,Government,Federal government,State government,Local government
1,2003-10-01,0.000000,0.001222,0.005236,0.000147,-0.000907,-0.000054,0.001253,0.002206,-0.000175,-0.002219,-0.001236,0.001364,0.003482,0.004160,-0.000185,0.002505,-0.002553,0.001404,0.003905
2,2003-11-01,0.000000,0.000606,0.005208,0.001766,-0.001257,0.000379,-0.001610,0.000838,-0.002625,-0.001587,-0.000866,0.002350,0.002120,0.002035,0.000370,-0.001068,-0.005132,0.000200,-0.000724
3,2003-12-01,0.000000,0.000935,-0.001739,0.004541,-0.001119,0.002198,-0.000168,-0.003172,-0.001226,0.000000,-0.000867,0.001667,0.002233,0.002922,-0.000185,0.000511,0.004380,-0.001405,0.000434
4,2004-01-01,0.000000,0.001630,0.001736,0.003067,-0.000700,0.002049,0.002884,0.007181,-0.001052,-0.002865,0.000248,0.001787,0.001877,0.002106,0.000370,-0.000371,-0.002928,-0.003223,0.001156
5,2004-02-01,0.000000,0.000412,0.001733,-0.001462,-0.000770,0.001024,-0.000060,0.001096,-0.002638,0.000000,0.000000,0.000801,0.001055,0.002102,0.000000,0.000557,-0.000366,0.000403,0.000794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2023-05-01,0.001802,0.001912,0.004666,0.003154,-0.000308,0.001272,0.001344,0.002760,0.001257,-0.001294,0.001423,0.001955,0.003487,0.001691,0.002220,0.001149,0.003083,0.002684,0.000207
237,2023-06-01,0.000673,0.000645,-0.001558,0.003646,0.000308,-0.001108,-0.001462,-0.002842,0.001435,-0.003246,0.000219,-0.000043,0.003121,0.001568,0.001024,0.000839,0.002051,0.002868,-0.000138
238,2023-07-01,0.001510,0.001086,0.003106,0.001506,-0.000154,0.002540,0.000811,-0.001013,-0.002698,-0.006205,0.001639,-0.001262,0.004091,0.002286,0.000682,0.004002,0.003407,0.002289,0.004739
239,2023-08-01,0.001450,0.001323,0.000000,0.004498,0.000847,0.000000,0.000026,-0.002824,0.004298,-0.006906,0.000218,0.000478,0.003801,0.002640,0.002381,0.002194,0.003734,0.004557,0.001029


In [20]:
covid_lockdown = employ_diff[197:203]
covid_lockdown

,Month,Total nonfarm,Total private,Mining and logging,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Utilities,Information,Financial activities,Professional and business services,Private education and health services,Leisure and hospitality,Other services,Government,Federal government,State government,Local government
198,2020-03-01,-0.009454,-0.010905,-0.014793,-0.009152,-0.005110,-0.002484,-0.005761,0.003376,-0.000365,-0.003105,-0.001920,-0.003934,-0.009116,-0.046117,-0.015385,-0.001313,0.007302,-0.009685,0.000000
199,2020-04-01,-0.157280,-0.180640,-0.092084,-0.155048,-0.114421,-0.071314,-0.164270,-0.098054,-0.009034,-0.098106,-0.030621,-0.113876,-0.119621,-0.859701,-0.296543,-0.041684,0.001389,-0.029119,-0.055205
200,2020-05-01,0.019729,0.028265,-0.042088,0.068769,0.019921,0.002871,0.028257,0.003844,-0.000369,-0.011494,0.001859,0.010019,0.018101,0.124183,0.048302,-0.024818,-0.000695,-0.011265,-0.035072
201,2020-06-01,0.033171,0.039187,-0.025907,0.022455,0.027636,0.006500,0.055503,0.023514,-0.005005,0.004577,0.001972,0.018749,0.024441,0.153905,0.069114,0.000514,0.002080,0.000198,0.000297
202,2020-07-01,0.010384,0.011416,-0.019366,0.003475,0.003412,-0.003188,0.015174,0.010054,-0.000556,-0.001911,0.000000,0.009636,0.008428,0.041351,0.026753,0.004742,0.007912,0.001972,0.005099
203,2020-08-01,0.012323,0.010380,-0.014286,0.004152,0.002159,0.002872,0.019669,0.017923,0.003880,0.009463,0.002891,0.009199,0.008786,0.022405,0.010214,0.022807,0.079481,0.008796,0.014996


In [21]:
employ_diff.drop(np.arange(198, 202), axis=0, inplace = True)

In [22]:
employ_diff.reset_index(drop = True, inplace = True)

In [24]:
employ_diff[196:200]

,Month,Total nonfarm,Total private,Mining and logging,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Utilities,Information,Financial activities,Professional and business services,Private education and health services,Leisure and hospitality,Other services,Government,Federal government,State government,Local government
196,2020-02-01,0.001792,0.001429,-0.002915,0.003680,-0.000391,-0.001018,-0.000561,-0.003802,0.000731,0.002407,0.003495,0.000606,0.002116,0.005429,0.000842,0.003848,-0.000350,0.008651,0.002925
197,2020-07-01,0.010384,0.011416,-0.019366,0.003475,0.003412,-0.003188,0.015174,0.010054,-0.000556,-0.001911,0.000000,0.009636,0.008428,0.041351,0.026753,0.004742,0.007912,0.001972,0.005099
198,2020-08-01,0.012323,0.010380,-0.014286,0.004152,0.002159,0.002872,0.019669,0.017923,0.003880,0.009463,0.002891,0.009199,0.008786,0.022405,0.010214,0.022807,0.079481,0.008796,0.014996
199,2020-09-01,0.006779,0.008787,-0.001789,0.004272,0.004710,0.004888,0.002714,0.012087,0.002947,0.020756,0.003343,0.005931,0.003837,0.037378,0.008811,-0.004197,-0.011531,-0.004122,-0.002554


In [25]:
employ_diff.to_csv('BLS_EmployDelta.csv')

# Simple Predictions

In [ ]:
employ_diff = pd.read_csv('BLS_EmployDelta.csv')

In [ ]:
employ_diff.info()

In [ ]:
employ_diff

In [ ]:
plt.close()
plt.plot(employ_diff['Month'], employ_diff['Construction'])
plt.show()

## Check for seasonality trend:

In [ ]:
seasonal = STL(employ_diff['Construction'], period=12)
res = seasonal.fit()
fig = res.plot()

I don't observe any particularly notable yearly trends!

## Try simple regression fit